In [91]:
# Importowanie bibliotek
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [92]:
# Wczytanie danych
data = pd.read_csv("D:\SAGES\project\Energy_consumption.csv")
data


,Timestamp,Temperature,Humidity,SquareFootage,Occupancy,HVACUsage,LightingUsage,RenewableEnergy,DayOfWeek,Holiday,EnergyConsumption
0,2022-01-01 00:00:00,25.139433,43.431581,1565.693999,5,On,Off,2.774699,Monday,No,75.364373
1,2022-01-01 01:00:00,27.731651,54.225919,1411.064918,1,On,On,21.831384,Saturday,No,83.401855
2,2022-01-01 02:00:00,28.704277,58.907658,1755.715009,2,Off,Off,6.764672,Sunday,No,78.270888
3,2022-01-01 03:00:00,20.080469,50.371637,1452.316318,1,Off,On,8.623447,Wednesday,No,56.519850
4,2022-01-01 04:00:00,23.097359,51.401421,1094.130359,9,On,Off,3.071969,Friday,No,70.811732
...,...,...,...,...,...,...,...,...,...,...,...
995,2022-02-11 11:00:00,28.619382,48.850160,1080.087000,5,Off,Off,21.194696,Saturday,No,82.306692
996,2022-02-11 12:00:00,23.836647,47.256435,1705.235156,4,Off,On,25.748176,Tuesday,Yes,66.577320
997,2022-02-11 13:00:00,23.005340,48.720501,1320.285281,6,Off,On,0.297079,Friday,Yes,72.753471
998,2022-02-11 14:00:00,25.138365,31.306459,1309.079719,3,On,Off,20.425163,Thursday,Yes,76.950389


In [93]:
# Wyświetlenie podstawowych informacji o danych
print("Podstawowe informacje o danych:")
print(data.info())
print("\nPodgląd danych:")
print(data.head())

Podstawowe informacje o danych:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Timestamp          1000 non-null   object 
 1   Temperature        1000 non-null   float64
 2   Humidity           1000 non-null   float64
 3   SquareFootage      1000 non-null   float64
 4   Occupancy          1000 non-null   int64  
 5   HVACUsage          1000 non-null   object 
 6   LightingUsage      1000 non-null   object 
 7   RenewableEnergy    1000 non-null   float64
 8   DayOfWeek          1000 non-null   object 
 9   Holiday            1000 non-null   object 
 10  EnergyConsumption  1000 non-null   float64
dtypes: float64(5), int64(1), object(5)
memory usage: 86.1+ KB
None

Podgląd danych:
             Timestamp  Temperature   Humidity  SquareFootage  Occupancy  \
0  2022-01-01 00:00:00    25.139433  43.431581    1565.693999          5   
1

In [94]:
# Eksploracja danych
print("\nSprawdzanie braków danych:")
print(data.isnull().sum())


Sprawdzanie braków danych:
Timestamp            0
Temperature          0
Humidity             0
SquareFootage        0
Occupancy            0
HVACUsage            0
LightingUsage        0
RenewableEnergy      0
DayOfWeek            0
Holiday              0
EnergyConsumption    0
dtype: int64


In [95]:
# Wypełnianie braków danych (jeśli występują)
#data.fillna(data.median(), inplace=True)

In [96]:
le = LabelEncoder()

for col in ['HVACUsage','LightingUsage', 'DayOfWeek', 'Holiday']:
    le.fit(data[col])
    data[col] = le.transform(data[col])

data.head()

,Timestamp,Temperature,Humidity,SquareFootage,Occupancy,HVACUsage,LightingUsage,RenewableEnergy,DayOfWeek,Holiday,EnergyConsumption
0,2022-01-01 00:00:00,25.139433,43.431581,1565.693999,5,1,0,2.774699,1,0,75.364373
1,2022-01-01 01:00:00,27.731651,54.225919,1411.064918,1,1,1,21.831384,2,0,83.401855
2,2022-01-01 02:00:00,28.704277,58.907658,1755.715009,2,0,0,6.764672,3,0,78.270888
3,2022-01-01 03:00:00,20.080469,50.371637,1452.316318,1,0,1,8.623447,6,0,56.519850
4,2022-01-01 04:00:00,23.097359,51.401421,1094.130359,9,1,0,3.071969,0,0,70.811732


In [97]:
# Wybór cech i zmiennej docelowej
# Zakładam, że kolumna 'Energy' zawiera zapotrzebowanie na energię (docelowa),
# a pozostałe kolumny to cechy
X = data.drop(columns=['EnergyConsumption','Timestamp'])  # Cechy
y = data['EnergyConsumption']                # Zmienna docelowa
X

,Temperature,Humidity,SquareFootage,Occupancy,HVACUsage,LightingUsage,RenewableEnergy,DayOfWeek,Holiday
0,25.139433,43.431581,1565.693999,5,1,0,2.774699,1,0
1,27.731651,54.225919,1411.064918,1,1,1,21.831384,2,0
2,28.704277,58.907658,1755.715009,2,0,0,6.764672,3,0
3,20.080469,50.371637,1452.316318,1,0,1,8.623447,6,0
4,23.097359,51.401421,1094.130359,9,1,0,3.071969,0,0
...,...,...,...,...,...,...,...,...,...
995,28.619382,48.850160,1080.087000,5,0,0,21.194696,2,0
996,23.836647,47.256435,1705.235156,4,0,1,25.748176,5,1
997,23.005340,48.720501,1320.285281,6,0,1,0.297079,0,1
998,25.138365,31.306459,1309.079719,3,1,0,20.425163,4,1


In [98]:
# Skalowanie cech
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [99]:
# Podział danych na zestawy treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [100]:
# Trenowanie modelu
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [101]:
# Ocena modelu na zbiorze testowym
y_pred = model.predict(X_test)

In [102]:
# Obliczenie metryk
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

In [103]:
print("\nWyniki modelu:")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")


Wyniki modelu:
Mean Absolute Error (MAE): 4.43
Mean Squared Error (MSE): 30.14
Root Mean Squared Error (RMSE): 5.49


In [104]:
# Opcjonalnie: zapisanie modelu lub predykcji
output = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
output.to_csv('energy_predictions.csv', index=False)
print("\nPredykcje zapisane do pliku 'energy_predictions.csv'")


Predykcje zapisane do pliku 'energy_predictions.csv'
